
<img src="../../img/ods_stickers.jpg">

## <center> [mlcourse.ai](https://mlcourse.ai) – открытый курс OpenDataScience по машинному обучению 
    
Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_predict

In [2]:
train = pd.read_csv("../data/flight_delays_train.csv")
test = pd.read_csv("../data/flight_delays_test.csv")

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


### Подготовка данных

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
# создаем признак маршрут

train['Route'] = train['Origin'] + '-' + train['Dest']
test['Route'] = test['Origin'] + '-' + test['Dest']

In [6]:
# one hot encoding

columns_to_encode = ['Month', 'Origin', 'Dest', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Route']

train = pd.get_dummies(train, columns=columns_to_encode)
test = pd.get_dummies(test, columns=columns_to_encode)

In [7]:
# проверяем, совпадает ли число столбцов

train.columns.shape, test.columns.shape

((5082,), (5339,))

In [8]:
# добавляем пропущенные столбцы

missing_columns = set(test.columns) - set(train.columns)

y_column = train['dep_delayed_15min']

# заполняем пропуски
for col in missing_columns:
    train[col] = 0  
    
# изменяем порядок
train = train[test.columns]
train['dep_delayed_15min'] = y_column

C:\Users\Acer\AppData\Local\Temp\ipykernel_15388\2071631615.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train[col] = 0


In [9]:
# преобразуем ответы

train["dep_delayed_15min"] = train["dep_delayed_15min"].map({"Y": 1, "N": 0})

In [10]:
X_train, y_train = (
    train.drop(columns=['dep_delayed_15min']).values,
    train["dep_delayed_15min"].values,
)

X_test = test.values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train, y_train, test_size=0.3, random_state=17
)

### Логистическая регрессия

In [11]:
lr = LogisticRegression(random_state=17)
lr.fit(X_train_part, y_train_part)

LogisticRegression(random_state=17)

In [39]:
lr_valid_pred = lr.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, lr_valid_pred)

0.6864526647118743

### Градиентный бустинг

In [12]:
# построим xgboost 

param_grid = {
    'max_depth': [5, 6, 7],
    'learning_rate': [0.001],
    'n_estimators': [100, 500, 1000],
}

In [13]:
xgb_model = xgb.XGBClassifier(objective='binary:logistic', random_state=17)
grid_search = GridSearchCV(xgb_model, param_grid, scoring='roc_auc', verbose=10, cv=5)

In [22]:
# ищем лучшие параметры

grid_search.fit(X_train_part, y_train_part)

best_params = grid_search.best_params_
best_params

{'learning_rate': 0.001, 'max_depth': 6, 'n_estimators': 100}

In [23]:
%%time

xgb_model = xgb.XGBClassifier(**best_params)
xgb_model.fit(X_train_part, y_train_part, verbose=10);

In [65]:
xgb_valid_pred = xgb_model.predict_proba(X_valid)[:, 1]
roc_auc_score(y_valid, xgb_valid_pred)

0.7329113755100138

### Комбинация двух моделей

In [19]:
def blend_probabilities(w1: float, prob1: np.array, prob2: np.array) -> np.array:
    return w1 * prob1 + (1 - w1) * prob2

def find_best_param(xgboost_model: XGBClassifier, logistic_model: LogisticRegression,
                    X_valid: np.array, y_valid: np.array, ws: list, cv=5):
    best_roc_auc = float("-inf")
    best_w = None
    for w in ws:
#         xgboost_probabilities = cross_val_predict(xgboost_model, X_valid, y_valid, cv=cv, method='predict_proba')[:, 1]
        xgboost_probabilities = xgboost_model.predict_proba(X_valid)[:, 1]
#         logistic_probabilities = cross_val_predict(logistic_model, X_valid, y_valid, cv=cv, method='predict_proba')[:, 1]
        logistic_probabilities = lr.predict_proba(X_valid)[:, 1]
        preds = blend_probabilities(w, xgboost_probabilities, logistic_probabilities)
        current_roc_auc = roc_auc_score(y_valid, preds)
        print(f"w's value: {w}; current ROC AUC: {current_roc_auc}")
        if current_roc_auc > best_roc_auc:
            best_roc_auc = current_roc_auc
            best_w = w
            print(f"New best w's value: {w}")

In [105]:
ws = np.linspace(0, 1, 11)

find_best_param(xgb_model, lr, X_valid, y_valid, ws)

w's value: 0.0; current ROC AUC: 0.686626735416265
New best w's value: 0.0
w's value: 0.1; current ROC AUC: 0.6970347954622074
New best w's value: 0.1
w's value: 0.2; current ROC AUC: 0.7060137667244376
New best w's value: 0.2
w's value: 0.30000000000000004; current ROC AUC: 0.7131850923896023
New best w's value: 0.30000000000000004
w's value: 0.4; current ROC AUC: 0.7188163453240124
New best w's value: 0.4
w's value: 0.5; current ROC AUC: 0.7232513363863591
New best w's value: 0.5
w's value: 0.6000000000000001; current ROC AUC: 0.7266921506603454
New best w's value: 0.6000000000000001
w's value: 0.7000000000000001; current ROC AUC: 0.7292956454577915
New best w's value: 0.7000000000000001
w's value: 0.8; current ROC AUC: 0.7311545177629626
New best w's value: 0.8
w's value: 0.9; current ROC AUC: 0.7323424667197769
New best w's value: 0.9
w's value: 1.0; current ROC AUC: 0.7329113755100138
New best w's value: 1.0


Т.к. градиент изначально предсказал лучше, то чем меньше вес регрессии, то тем лучше, судя по полученным результатам.

В дальнейшем возьмем $w = 0.7$.

### Предсказания

In [45]:
# только для логистической регрессии

lr_test_pred = lr.predict_proba(X_test)[:, 1]

pd.Series(lr_test_pred, name="dep_delayed_15min").to_csv(
    "lr_test_pred.csv", index_label="id", header=True
)

# Kaggle score: 0.69081

In [20]:
# только для градиентного бустинга

xgb_test_pred = xgb_model.predict_proba(X_test)[:, 1]

pd.Series(xgb_test_pred, name="dep_delayed_15min").to_csv(
    "xgb_test_pred.csv", index_label="id", header=True
)

# Kaggle score: 0.72564

In [60]:
# для комбинации моделей
w = 0.7

lr_test_pred = lr.predict_proba(X_test)[:, 1]
xgb_test_pred = xgb_model.predict_proba(X_test)[:, 1]
lr_xgb_pred = blend_probabilities(0.7, xgb_test_pred, lr_test_pred)

pd.Series(lr_xgb_pred, name="dep_delayed_15min").to_csv(
    "lr_xgb_pred.csv", index_label="id", header=True
)

# Kaggle score: 0.72468

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!